# Find clear scenes in high-resolution SST (local computer)

### What I want to do with this notebook is to choose a target region and time period, cycle through a bunch of high-resolution sea surface temperature data in the PO.DAAC cloud, identify clear images, catalog them

@jtomfarrar

This was modified from this notebook:
https://github.com/NASA-Openscapes/nch2021-projects-contexdata/blob/main/notebooks/find_VIIRS_SST.ipynb

The major change I want to make is to run the notebook locally on my computer, rather than on the Amazon Cloud.

- We will use L3 VIIRS SST here, PO.DAAC short name "VIIRS_NPP-OSPO-L3U-v2.61"


Issues/to-do:

1. Processing a year of data consumes all the memory on a small cloud instance (when xr.open_mfdataset() is called)
1. It seems like I am hitting a limit of the number of granules that can be returned by using requests.get to query 'https://cmr.earthdata.nasa.gov/search/granules'
1. It would be nice to print/save a list of clear dates

## Summary of what I learned 

1. Cloud computing is just using someone else’s computer.
1. The place where there is an advantage to be had is when you can do cloud computing next to a huge amount of data
1. If you must work locally, or on a different cloud than where the data are, data sets in the cloud are functionally just data sets on the web
1. We can query the NASA Common Metadata Repository to find out the names and “concept ID” of datasets (url = 'https://cmr.earthdata.nasa.gov/search/collections.umm_json'); see example below in this notebook
1. To download data (to cloud or local machine), you need a _netrc file (on windows) or .netrc file (on linux) to pass NASA EarthData credentials and get S3 access;—see Tutorial 4, https://github.com/NASA-Openscapes/2021-Cloud-Hackathon/blob/main/tutorials/04_NASA_Earthdata_Authentication.ipynb ; this is needed in “def begin_s3_direct_access()” below; for direct download, it would also be needed
1. We were working in the Amazon Cloud using the “2i2c Hub Service”:
https://openscapes.2i2c.cloud/hub/login?next=%2Fhub%2F


In [1]:
#Pretty sure some of these are extraneous
import s3fs
import os
import os.path
import json
import warnings
import requests
import numpy as np
import pandas as pd
import xarray as xr
from io import StringIO

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import FormatStrFormatter
from matplotlib import ticker, rc, cm

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

# import geoviews as gv
# import hvplot.xarray
# import holoviews as hv
# gv.extension('bokeh', 'matplotlib')

warnings.filterwarnings("ignore")

%matplotlib inline
# %matplotlib qt5

## Set some parameters (start/end time)


In [2]:
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 700

## PO DAAC Catalog exploration using the CMR API

- The search is restricted to colections that have data over a given time window

In [3]:
provider = 'POCLOUD'
url = 'https://cmr.earthdata.nasa.gov/search/collections.umm_json'
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'page_size': 500,
                            'temporal': ['2021-10-01T10:00:00Z','2021-11-01T00:00:00Z'] 
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )
response.headers['cmr-hits']

'145'

### Loop through each response in the catalog and print the respective concept ID

These are the data sets that cover the time period of interest

In [4]:
#  If set to "True", this will show short names and concept ids for all PO.DAAC data sets at this time
if False:
    for r in response.json()['items']:
        print('{} ==> '.format(r['meta']['s3-links'][0].split('/')[1]), r['meta']['concept-id'])

### Loading credentials for direct access

This seems to only work from a cloud machine.  Is there a substitute for access from my local machine?

In [5]:
def begin_s3_direct_access(url: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})

fs = begin_s3_direct_access()

type(fs)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

For local access, let's try "PO.DAAC data subscriber":
https://github.com/podaac/data-subscriber

In [5]:
podaac-data-subscriber -c CYGNSS_L1_CDR_V1.0 -d myData

SyntaxError: invalid syntax (<ipython-input-5-dff4f2cefbc4>, line 1)

In [6]:
# Center location
site = 'S-MODE'

if site == 'S-MODE':
    lon0 = -123.5
    lat0 = 37.5 
    dlon = 2.5 # half of box width in lon
    dlat = 1.5 # half of box width in lat
elif site == 'SPURS-2':
    lon0 = -125
    lat0 = 10
    dlon = 1.5 # half of box width in lon
    dlat = 1.5 # half of box width in lat
elif site == 'SPURS-2':
    lon0 = -38
    lat0 = 25
    dlon = 1.5 # half of box width in lon
    dlat = 1.5 # half of box width in lat

# Define the max/min lon
lon_min = lon0 - dlon
lon_max = lon0 + dlon
lat_min = lat0 - dlat
lat_max = lat0 + dlat

In [7]:
if site == 'S-MODE':
    start_time = '2021-10-20T00:00:00Z'
    end_time = '2021-11-6T00:00:00Z'
elif site == 'SPURS-2':
    start_time = '2016-8-20T00:00:00Z'
    end_time = '2017-11-6T00:00:00Z'
elif site == 'SPURS-2':
    start_time = '2012-9-15T00:00:00Z'
    end_time = '2013-11-15T00:00:00Z'



In [8]:
url = 'https://cmr.earthdata.nasa.gov/search/granules'

# VIIRS_NPP-OSPO-L2P-v2.61 ==>  C1996880725-POCLOUD
concept_id = 'C2036880650-POCLOUD' #  MODIS
# concept_id = 'C1996880725-POCLOUD' # VIIRS_NPP-OSPO-L2P-v2.61 
concept_id = 'C2036877595-POCLOUD' # VIIRS_NPP-OSPO-L3U-v2.61 
response = requests.get(url, 
                        params={
                            'concept_id': concept_id,
                            'temporal': start_time+','+end_time,
                            'bounding_box': '{},{},{},{}'.format(lon_min, lat_min, lon_max, lat_max),    
                            'page_size': 500,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response)
print(response.headers['CMR-Hits'])

<Response [200]>
94


In [9]:
granules_url = []
for gran in response.json()['feed']['entry']:
    granules_url.append(gran['links'][0]['href'])
# granules_url
len(granules_url)

94

In [10]:
gran

{'boxes': ['7.88 -150.44 46.61 -108.77'],
 'time_start': '2021-11-05T21:30:00.000Z',
 'updated': '2021-11-05T22:21:13.722Z',
 'dataset_id': 'GHRSST Level 3U OSPO dataset v2.61 from VIIRS on S-NPP Satellite (GDS v2)',
 'data_center': 'POCLOUD',
 'title': '20211105213000-OSPO-L3U_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0',
 'coordinate_system': 'CARTESIAN',
 'day_night_flag': 'UNSPECIFIED',
 'time_end': '2021-11-05T21:39:59.000Z',
 'id': 'G2159445200-POCLOUD',
 'original_format': 'UMM_JSON',
 'granule_size': '4.286630630493164',
 'browse_flag': False,
 'collection_concept_id': 'C2036877595-POCLOUD',
 'online_access_flag': True,
 'links': [{'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#',
   'title': 'This link provides direct download access via S3 to the granule.',
   'hreflang': 'en-US',
   'href': 's3://podaac-ops-cumulus-protected/VIIRS_NPP-OSPO-L3U-v2.61/20211105213000-OSPO-L3U_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc'},
  {'rel': 'http://esipfed.org/ns/fe

## Plot all SST images available during the S-MODE windows and pick good days

Load in the data with xarray.open_mfdataset().  The one trick for the s3 access here is to use list comprehension to modify the list of granule URLs into a list of open s3 files before passing to xr.open_mfdataset().  

_(Is that description correct??)_

In [ ]:
# This loads a single file
# ds = xr.open_dataset(fs.open(granules_url[1]),drop_variables=['dt_analysis','satellite_zenith_angle','sses_bias','wind speed']).sel(lon=slice(lon_min,lon_max), lat=slice(lat_max,lat_min))

In [ ]:
file_list =  [fs.open(file) for file in granules_url]
ds = xr.open_mfdataset(file_list,drop_variables=['dt_analysis','satellite_zenith_angle','sses_bias','wind speed']).sel(lon=slice(lon_min,lon_max), lat=slice(lat_max,lat_min))

NameError: name 'fs' is not defined

In [ ]:
#  Plot the whole timeseries with hvplot

sst = ds.sea_surface_temperature-273.15
sst.hvplot(groupby='time', width=600, height=400, clim=(10, 20), cmap='plasma')

As expected, there are lots of times with no useful data.  (There is an interactive plot above, but it may not render in a static view.)

## Make a metric to select times with clear skies in region of interest

The idea here is to choose a box defining region of interest and use quality flag or NaN mask to count bad/good pixels


In [ ]:
# Define a box where we want data (may be different than larger analysis domain)
# Still centered on lon0,lat0
dlon = 0.5 # half of box width in lon
dlat = 0.5 # half of box width in lat

# Define the max/min lon
x1 = lon0 - dlon
x2 = lon0 + dlon
y1 = lat0 - dlat
y2 = lat0 + dlat

In [ ]:
# Make a time series in that box
# Compute mean value of "not NaN" (notnull) in the box
good_data = ds.sea_surface_temperature.sel(lat=slice(y2,y1), lon=slice(x1,x2)).notnull().mean({'lon','lat'})
np.shape(good_data)

In [ ]:
fig = plt.figure(figsize=(4,3),dpi=200)
plt.plot(good_data['time'],good_data,marker='o')
fig.autofmt_xdate()
plt.title('Values near 1 indicate clear skies')

In [ ]:
good_times = good_data['time'].where(good_data>0.95,drop=True)
np.shape(good_times)

In [ ]:
good_times[-1].values

## Choose a particular time


In [ ]:
# Find the data near this time
d = good_times[-1] # np.datetime64('2021-11-05T00:00:00Z')

In [ ]:
ds2 = ds.sel(time=d,method='nearest')

In [ ]:
# This string of the time of the SST selected will be useful
day_str = np.datetime_as_string(ds2.time,unit='m')

In [ ]:
fig = plt.figure()
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=-125))  # Orthographic
extent = [lon_min, lon_max,lat_min, lat_max]
ax.set_extent(extent, crs=ccrs.PlateCarree())


plt.set_cmap(cmap=plt.get_cmap('turbo'))
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
ax.set_title('SST, ' + day_str, size = 10.)

ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')
cs = ax.pcolormesh(ds2.lon,ds2.lat,ds2.sea_surface_temperature.squeeze()-274.15,vmin=12.5,vmax=15,transform=ccrs.PlateCarree())
# cs = ax.pcolormesh(ds2.lon,ds2.lat,ds2.sea_surface_temperature.squeeze().notnull(),transform=ccrs.PlateCarree())
cb = plt.colorbar(cs,fraction = 0.022,extend='both')
cb.set_label('SST [$\circ$C]',fontsize = 10)
